<a href="https://colab.research.google.com/github/Yomex139/NLP/blob/main/NLP_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import transformers

In [ ]:
transformers.__version__
import pandas as pd

In [ ]:

from huggingface_hub import notebook_login

In [ ]:

notebook_login()

In [ ]:

from datasets import load_dataset

In [ ]:

dataset = load_dataset('rotten_tomatoes')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [ ]:

classifier = transformers.pipeline('sentiment-analysis', truncation=True, max_length=512)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
classifier_new = transformers.pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def predict(dataset):
  return classifier(dataset)[0]['label']

dframe = dataset['test'].to_pandas()
dframe['prediction'] = dframe['text'].apply(predict)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
import numpy as np
dframe['prediction'] = dframe['prediction'].apply(lambda x: x.lower()).map({'negative':0, 'positive':1})
dframe

,text,label,prediction
0,lovingly photographed in the manner of a golde...,1,1
1,consistently clever and suspenseful .,1,1
2,"it's like a "" big chill "" reunion of the baade...",1,0
3,the story gives ample opportunity for large-sc...,1,1
4,"red dragon "" never cuts corners .",1,1
...,...,...,...
1061,a terrible movie that some people will neverth...,0,0
1062,there are many definitions of 'time waster' bu...,0,0
1063,"as it stands , crocodile hunter has the hurrie...",0,0
1064,the thing looks like a made-for-home-video qui...,0,0


In [ ]:

review = load_dataset('jahjinx/IMDb_movie_reviews')

README.md: 0.00B [00:00, ?B/s]

IMDB_train.csv:   0%|          | 0.00/47.6M [00:00<?, ?B/s]

IMDB_validation.csv: 0.00B [00:00, ?B/s]

IMDB_test.csv:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/36000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:

def score(review):
  return classifier(review)[0]['label']
df_data = review['test'].to_pandas()

In [ ]:


df_data['predictions'] = df_data['text'].apply(score)

In [ ]:


df_data['predictions'] = df_data['predictions'].apply(lambda x: x.lower()).map({'negative': 0, 'positive': 1})

In [ ]:


df_data.to_csv('predictions.csv')

In [ ]:


pd.read_csv('predictions.csv')

,Unnamed: 0,text,label,predictions
0,0,With No Dead Heroes you get stupid lines like ...,0,0
1,1,I thought maybe... maybe this could be good. A...,0,0
2,2,An elite American military team which of cours...,0,0
3,3,Ridiculous horror film about a wealthy man (Jo...,0,0
4,4,"Well, if you are one of those Katana's film-nu...",1,1
...,...,...,...,...
9995,9995,I liked this movie. That's pretty much all I c...,1,1
9996,9996,"Imagine the plight of Richard, a painter, whos...",1,1
9997,9997,There is an inherent problem with commenting o...,0,0
9998,9998,When The Matrix appeared in 1999 and questione...,1,1


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(df_data['label'], df_data['predictions']))
print(classification_report(df_data['label'], df_data['predictions']))


Accuracy: 0.8874
              precision    recall  f1-score   support

           0       0.86      0.92      0.89      5044
           1       0.91      0.85      0.88      4956

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [ ]:
texts = df_data['text'].to_list()
prediction = classifier_new(texts, truncation=True, max_length=512, batch_size=32)

In [ ]:
df_data['prediction'] = [1 if x['label'].lower() == 'positive' else 0 for x in prediction]

In [ ]:
df_data

,text,label,predictions,prediction
0,With No Dead Heroes you get stupid lines like ...,0,0,0
1,I thought maybe... maybe this could be good. A...,0,0,0
2,An elite American military team which of cours...,0,0,0
3,Ridiculous horror film about a wealthy man (Jo...,0,0,0
4,"Well, if you are one of those Katana's film-nu...",1,1,0
...,...,...,...,...
9995,I liked this movie. That's pretty much all I c...,1,1,0
9996,"Imagine the plight of Richard, a painter, whos...",1,1,0
9997,There is an inherent problem with commenting o...,0,0,0
9998,When The Matrix appeared in 1999 and questione...,1,1,0


In [ ]:
from transformers import pipeline
pipe = pipeline('summarization', model='Falconsai/text_summarization')

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
words = "The silence of space was broken only by the steady hum of the engines as the shuttle Odyssey drifted beyond Earth’s orbit, a gleaming arrow of technology and human ambition cutting through the void, carrying with it not only seven astronauts but also the collective hopes of millions who had watched its launch on glowing screens back home, and as the Earth grew smaller behind them, Commander Elena Ramirez thought about the fragile blue planet they were leaving, reflecting on the generations of engineers, dreamers, and explorers who had made this moment possible, because this mission was not like others before it—it was not a simple deployment of satellites or a short visit to the International Space Station but rather humanity’s first attempt to establish a forward base on the far side of the Moon, a staging ground for the even greater leap to Mars, and the responsibility weighed heavily on her shoulders, though she wore her calm exterior like a shield, hiding from her crew the storm of anticipation that churned within her, and as she glanced around the cockpit she saw her team engaged in their duties with the quiet professionalism that comes only after years of training, each astronaut a master of their craft: Dr. Anaya Patel, the mission’s lead scientist, reviewing the biological samples stored in stasis that would be studied for long-term survival in lunar conditions; Lieutenant Marcus Shaw, the pilot, his steady hands guiding the shuttle with precision honed through endless simulations; Engineer Yuri Volkov, a stoic Russian whose genius with machinery had already saved many missions during test flights; and specialists Mei Lin, Carlos Mendes, and Jamal Reed, each responsible for communications, navigation, and medical research respectively, all aware that failure was not an option because the eyes of the world were fixed upon them, and as the shuttle pushed farther from Earth, the systems began to record minute fluctuations in radiation levels, subtle warnings that space was not a friendly environment, yet Elena reminded herself that danger had always been the companion of discovery, from the sailors who braved stormy oceans centuries ago to the aviators who first pierced the stratosphere, and this journey was simply the next chapter in that long, unbroken story of courage, and when they finally approached lunar orbit, the sight that greeted them was breathtaking: the far side of the Moon, a landscape humanity had seen only through robotic probes, now revealed in stark, unforgiving beauty, with jagged craters and shadowed valleys stretching endlessly beneath them like the scars of ancient battles, and Yuri’s voice crackled in the cabin as he announced that the automated systems were preparing for descent, each movement of the shuttle carefully choreographed to avoid disaster, and the landing was a tense ballet of thrusters firing, dust plumes rising, and instruments beeping until at last the Odyssey touched down with a shudder, the first human craft to rest on this remote, hidden face of the Moon, and cheers erupted from Mission Control thousands of miles away, though the astronauts themselves allowed only brief smiles before focusing on the monumental tasks ahead, because establishing a lunar base was no simple matter—it required precision, endurance, and above all teamwork, and so the crew began the slow process of unloading modules, assembling habitats, and deploying solar arrays, each step a careful dance with the hostile environment, where every mistake could mean catastrophe, and as days turned into weeks, the barren lunar soil was transformed into something extraordinary: a living outpost, small yet vital, a beacon of humanity’s determination to expand its horizons, but even as progress was made, challenges grew, for cosmic radiation posed risks to their health, micrometeorites peppered the shielding, and communication delays with Earth sometimes stretched beyond comfort, forcing the crew to rely more on their own judgment than on distant voices from mission control, and amidst all this, tensions simmered, as Marcus grew restless with the confinement, Mei felt isolated without real-time communication with her family, and Anaya wrestled with the ethical implications of her experiments, questioning whether life should be forced to adapt to such a hostile environment, yet Elena, ever the leader, found ways to guide them through, drawing upon the trust they had built during years of training, reminding them that their mission was larger than any one of them, that they were the tip of a spear thrusting into the unknown on behalf of all humankind, and then came the moment that would define their journey forever, when Jamal detected an anomaly beneath the lunar surface, a strange magnetic reading inconsistent with natural formations, and though some dismissed it as a sensor glitch, curiosity demanded investigation, and so they ventured beyond their base, trekking across the desolate landscape in their suits, until they found themselves standing before an immense cavern revealed by a recent meteor impact, its entrance yawning like the mouth of some ancient beast, and inside, their lights revealed structures—geometric patterns etched into the stone, too precise to be the product of geology, sparking debates that crackled across the crew, because if these formations were artificial, then humanity was not alone in the universe, and the implications were staggering, but before further analysis could be done, disaster struck in the form of a solar storm, a blast of radiation that swept across space, disabling satellites, frying circuits, and leaving the crew scrambling for survival as their systems faltered, the storm battering their fragile base like an invisible hurricane, and in the chaos Yuri was injured while repairing the shielding, his suit torn by shrapnel, forcing Mei and Jamal to drag him back to safety while Marcus struggled to stabilize the shuttle’s failing power grid, and Elena herself was nearly blinded when a panel exploded in her face, yet through sheer determination and the grit born of training, they weathered the storm, though not without scars, and in the aftermath, as they surveyed the damage and tended to Yuri’s wounds, they realized how thin the line was between triumph and tragedy, how easily their bold venture could collapse into catastrophe, but instead of retreating, they pressed forward, rebuilding, adapting, and choosing to continue, for to quit would dishonor the sacrifices already made, and slowly the systems were restored, the experiments resumed, and the crew returned to the cavern, this time with better equipment, determined to uncover its secrets, and what they found reshaped their understanding of humanity’s place in the cosmos: crystalline structures aligned with the stars, inscriptions that hinted at knowledge far beyond human science, relics of a civilization that perhaps had once visited the Moon or even seeded life on Earth itself, and though much remained uncertain, one truth was clear: humanity was no longer alone, and as they prepared to transmit their findings back to Earth, they knew that history had changed forever, that their journey would be remembered not just as the first lunar colony but as the moment humanity’s gaze turned outward with new urgency, driven by the knowledge that others had walked among the stars long before them, and as the shuttle’s engines roared to life for the return journey, Elena looked once more at the barren, beautiful surface of the Moon and whispered a promise to herself and to the universe: that humanity would not stop here, that the story of exploration was only beginning, and that the spirit of discovery, no matter the dangers, would carry them to Mars, to the stars, and beyond, because the destiny of humanity was not to remain bound to Earth but to rise, again and again, into the infinite night of space."

result = pipe(words, max_length=200, min_length=30, do_sample=False)
result[0]['summary_text']

Token indices sequence length is longer than the specified maximum sequence length for this model (1707 > 512). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'mission was not a simple deployment of satellites or a short visit to the International Space Station, but rather humanity’s first attempt to establish a forward base on the far side of the Moon . Lieutenant Marcus Shaw, the pilot, his steady hands guiding the shuttle with precision honed through endless simulations . Yuri Volkov, a stoic Russian whose genius with machinery had already saved many missions during test flights .'

In [ ]:
model = pipeline('ner', model="cahya/NusaBert-ner-v1.3", grouped_entities=True)
text = "landscape in their suits, until they found themselves standing before an immense cavern revealed by a recent meteor impact, its entrance yawning like the mouth of some ancient beast, and inside, their lights revealed structures—geometric patterns etched into the stone, too precise to be the product of geology, sparking debates that crackled across the crew, because if these formations were artificial, then humanity was not alone in the universe, and the implications were staggering, but before further analysis could be done, disaster struck in the form of a solar storm, a blast of radiation that swept across space, disabling satellites, frying circuits, and leaving the crew scrambling for survival as their systems faltered, the storm battering their fragile base like an invisible hurricane, and in the chaos Yuri was injured while repairing the shielding, his suit torn by shrapnel, forcing Mei and Jamal to drag him back to safety while Marcus struggled to stabilize the shuttle’s failing power grid, and Elena herself was nearly blinded when a panel exploded in her face, yet through sheer determination and the grit born of training, they weathered the storm, though not without scars, and in the aftermath, as they surveyed the damage and tended to Yuri’s wounds, they realized how thin the line was between triumph and tragedy, how easily their bold venture could collapse into catastrophe, but instead of retreating, they pressed forward, rebuilding, adapting, and choosing to continue, for to quit would dishonor the sacrifices already made, and slowly the systems were restored, the experiments resumed, and the crew returned to the cavern, this time with better equipment, determined to uncover its secrets, and what they found reshaped their understanding of humanity’s place in the cosmos: crystalline structures aligned with the stars, inscriptions that hinted at knowledge far beyond human science, relics of a civilization that perhaps had once visited the Moon or even seeded life on Earth itself, and though much remained uncertain, one truth was clear: humanity was no longer alone, and as they prepared to transmit their findings back to Earth, they knew that history had changed forever, that their journey would be remembered not just as the first lunar colony but as the moment humanity’s gaze turned outward with new urgency, driven by the knowledge that others had walked among the stars long before them, and as the shuttle’s engines roared to life for the return journey, Elena looked once more at the barren, beautiful surface of the Moon and whispered a promise to herself and to the universe: that humanity would not stop here, that the story of exploration was only beginning, and that the spirit of discovery, no matter the dangers, would carry them to Mars, to the stars, and beyond, because the destiny of humanity was not to remain bound to Earth but to rise, again and again, into the infinite night of space."

data_dict = model(text)

config.json:   0%|          | 0.00/2.64k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/644M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [ ]:
pd.DataFrame(data_dict)

,entity_group,score,word,start,end
0,PRD,0.575596,satellites,632,643
1,PER,0.999688,Yuri,818,823
2,PER,0.999945,Mei,901,905
3,PER,0.999852,Jamal,909,915
4,PER,0.999104,Marcus,948,955
5,PRD,0.676192,shuttle,982,990
6,PRD,0.665028,grid,1006,1011
7,PER,0.999535,Elena,1016,1022
8,PER,0.998936,Yuri,1263,1268
9,LOC,0.532432,Moon,1981,1986


In [ ]:
model_checkpoint = "consciousAI/question-answering-roberta-base-s-v2"
model1 = pipeline('question-answering', model=model_checkpoint)
contents = """In the year 2028, the Global Alliance for Sustainable Energy (GASE), formed by 42 countries, launched a massive initiative known as Project Helios. The project’s primary goal was to construct the first orbiting solar power station positioned at the Lagrange Point L1, nearly 1.5 million kilometers away from Earth. This station, named Aurelia, was designed to capture solar radiation continuously and transmit energy back to Earth using microwave beams. The construction required five space shuttle missions, the first led by Commander Aisha Rahman of Nigeria, who became the first African woman to command an international spaceflight. While Aisha’s crew assembled the primary solar arrays, another critical team on Earth, led by Dr. Miguel Serrano from Chile, worked on the ground-based receivers that would convert the microwave energy back into usable electricity. By late 2029, Aurelia successfully powered 12% of Western Europe’s energy grid during its testing phase.

However, Project Helios was not without controversy. Some environmental scientists raised concerns about microwave leakage potentially disrupting migratory bird patterns, while others argued that reliance on space-based energy could worsen global inequality if poorer nations lacked access to the technology. Interestingly, during the second shuttle mission in 2028, astronaut Yuri Tanaka from Japan discovered a flaw in the docking mechanism that, if left unresolved, could have caused a catastrophic failure. Her quick thinking saved the mission, and her actions were later recognized with the Global Science Medal.

By 2032, an independent report revealed that Project Helios had reduced global fossil fuel consumption by nearly 7%, preventing approximately 1.2 billion tons of carbon emissions. Still, geopolitical tension simmered as some countries accused the GASE coalition of monopolizing solar-rich orbital positions. Rumors even circulated that rival blocs were secretly planning to launch their own competing stations by 2035."""
question = "Which individual worked on ground-based receivers, and in which country were they located?"

result = model1(question=question, context=contents)

config.json:   0%|          | 0.00/782 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
result

{'score': 0.39938414096832275, 'start': 755, 'end': 760, 'answer': 'Chile'}

In [ ]:
# Load English → German translation pipeline
translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


In [ ]:
text = "Last Saturday, I went to the zoo with my family. The weather was bright and sunny. At the zoo, we saw many animals. The lions were resting under a tree, and the monkeys were jumping from branch to branch. We also saw tall giraffes eating leaves from the trees and colorful birds flying around. My favorite part was watching the elephants spray water with their trunks. We took many pictures and bought some snacks before going home. It was a fun and exciting day."

result = translator(text, max_length=200)

In [ ]:
result[0]['translation_text']

'Am vergangenen Samstag ging ich mit meiner Familie in den Zoo. Das Wetter war hell und sonnig. Im Zoo sahen wir viele Tiere. Die Löwen ruhten unter einem Baum, und die Affen sprangen von Ast zu Ast. Wir sahen auch große Giraffen, die Blätter von den Bäumen essen und bunte Vögel herumfliegen. Mein Lieblingsteil war es, die Elefanten Wasser mit ihren Stämmen zu sprühen. Wir machten viele Bilder und kauften einige Snacks, bevor wir nach Hause gingen. Es war ein lustiger und aufregender Tag.'

In [ ]:
zero_classifier = pipeline(task="zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
texts = """Recently, astronomers announced the discovery of a new exoplanet located approximately 120 light-years from Earth, orbiting a star remarkably similar to our Sun in size and luminosity. This planet, designated as Kepler-452c, resides within the so-called ‘habitable zone,’ where conditions may allow liquid water to exist on its surface. Observations suggest that it has a rocky composition and experiences a climate that could potentially support life forms, similar to those found on Earth. Scientists used advanced spectroscopy and space telescopes to analyze the planet’s atmosphere, detecting traces of gases such as oxygen and methane, which are often associated with biological processes. The discovery has sparked excitement in the scientific community, as it not only expands our understanding of planetary systems but also reignites the age-old question of whether humans are alone in the universe. Ongoing studies aim to determine the planet’s exact orbital period, surface temperature, and potential for sustaining life, and the findings could have profound implications for future interstellar exploration and the search for extraterrestrial civilizations."""
labels = ["Science", "Entertainment", "Politics", "Sports", "Technology", "Health"]

result = zero_classifier(texts, labels)

In [ ]:
result

{'sequence': 'Recently, astronomers announced the discovery of a new exoplanet located approximately 120 light-years from Earth, orbiting a star remarkably similar to our Sun in size and luminosity. This planet, designated as Kepler-452c, resides within the so-called ‘habitable zone,’ where conditions may allow liquid water to exist on its surface. Observations suggest that it has a rocky composition and experiences a climate that could potentially support life forms, similar to those found on Earth. Scientists used advanced spectroscopy and space telescopes to analyze the planet’s atmosphere, detecting traces of gases such as oxygen and methane, which are often associated with biological processes. The discovery has sparked excitement in the scientific community, as it not only expands our understanding of planetary systems but also reignites the age-old question of whether humans are alone in the universe. Ongoing studies aim to determine the planet’s exact orbital period, surface te

In [ ]:
texts = "Machine learning, often abbreviated as ML, is a subfield of artificial intelligence that focuses on designing algorithms and statistical models that enable computers to perform tasks without explicit instructions, relying instead on patterns and inference drawn from data, and its history can be traced back to the mid-20th century when pioneers such as Alan Turing, Arthur Samuel, and others began exploring the possibility of creating machines that could learn from experience, with Turing proposing the famous “Turing Test” as a measure of machine intelligence, and Samuel developing one of the earliest self-learning programs for playing checkers, laying the groundwork for decades of research that would eventually lead to the rich ecosystem of ML techniques we see today; over the years, machine learning has evolved through various paradigms including supervised learning, unsupervised learning, reinforcement learning, and semi-supervised learning, each of which addresses specific types of problems, for instance, supervised learning involves training a model on labeled data so that it can predict outcomes for unseen inputs, with algorithms ranging from linear regression, logistic regression, support vector machines, decision trees, random forests, gradient boosting machines, to more complex neural networks, while unsupervised learning focuses on discovering hidden patterns or intrinsic structures in data that are not labeled, using techniques such as clustering, principal component analysis, and autoencoders; reinforcement learning, inspired by behavioral psychology, allows agents to learn optimal strategies through interactions with an environment, receiving rewards or penalties based on their actions, and has been successfully applied in areas such as robotics, game playing, and autonomous vehicles, whereas semi-supervised learning leverages a small amount of labeled data together with a large pool of unlabeled data to improve model performance in scenarios where labeling is expensive or impractical; the applications of machine learning are vast and impact virtually every sector of modern life, from healthcare, where ML algorithms assist in disease diagnosis, personalized treatment plans, and predictive analytics, to finance, where models are employed for fraud detection, algorithmic trading, credit scoring, and risk management, as well as marketing, where customer segmentation, recommendation systems, sentiment analysis, and churn prediction are powered by ML techniques, and in transportation, autonomous vehicles rely on ML for perception, decision-making, and path planning, while in agriculture, machine learning models help optimize crop yields, detect plant diseases, and manage resources efficiently, and in entertainment, recommendation engines for movies, music, and other media are driven by sophisticated algorithms that analyze user behavior to predict preferences; the backbone of these applications is data, often referred to as the new oil, which is collected from diverse sources such as sensors, social media, transaction records, scientific experiments, medical imaging, satellite imagery, and IoT devices, and the quality, quantity, and representativeness of this data largely determine the success of machine learning models, making data preprocessing, cleaning, normalization, and augmentation critical steps in any ML pipeline, along with feature engineering, which transforms raw data into meaningful inputs that improve model performance, and hyperparameter tuning, which optimizes the settings of algorithms to achieve better accuracy, efficiency, and generalization; furthermore, the rise of deep learning, a subset of machine learning that leverages multi-layered artificial neural networks, has dramatically enhanced the capabilities of models to learn complex representations from unstructured data such as images, audio, and text, leading to breakthroughs in computer vision, natural language processing, speech recognition, and generative models, with architectures like convolutional neural networks (CNNs) excelling in image classification and object detection tasks, recurrent neural networks (RNNs) and their variants, including long short-term memory (LSTM) networks and gated recurrent units (GRUs), being highly effective for sequential data, and transformer-based models, such as BERT, GPT, and their successors, revolutionizing NLP by enabling contextual understanding and large-scale pretraining on massive corpora; machine learning also raises important ethical and societal considerations, including issues of bias, fairness, transparency, accountability, and privacy, as models trained on historical data may inadvertently perpetuate discrimination or inequity, and the deployment of autonomous decision-making systems in critical areas like healthcare, criminal justice, finance, and hiring requires careful oversight and regulatory frameworks to ensure responsible usage, alongside interpretability methods such as SHAP, LIME, and attention visualization that aim to explain model predictions to humans, and ongoing research into federated learning, differential privacy, and secure multi-party computation seeks to balance the benefits of ML with the need to protect sensitive information; the development of ML infrastructure, frameworks, and libraries, such as TensorFlow, PyTorch, Scikit-learn, XGBoost, and Hugging Face Transformers, has democratized access to powerful algorithms, enabling researchers, developers, and organizations to build sophisticated models without starting from scratch, while cloud computing platforms, GPUs, TPUs, and distributed computing systems provide the computational resources necessary to train and deploy large-scale models efficiently; education and knowledge sharing in the ML community, through open-source projects, academic publications, conferences, online courses, and collaborative competitions like Kaggle, have accelerated innovation and adoption, allowing both beginners and experts to contribute to advancing the field; looking to the future, machine learning continues to evolve rapidly, with emerging areas such as self-supervised learning, neuro-symbolic AI, continual learning, and quantum machine learning promising to tackle increasingly complex tasks, improve generalization, and enable AI systems that can reason, adapt, and interact with humans more naturally, ultimately transforming industries, scientific discovery, and society as a whole while posing profound philosophical questions about intelligence, creativity, and the relationship between humans and machines, making machine learning not only a technological revolution but also a lens through which we examine the possibilities and limitations of artificial intelligence in the 21st century and beyond, highlighting the necessity for interdisciplinary collaboration, ethical stewardship, and continuous learning to ensure that the benefits of machine learning are realized responsibly and inclusively across the globe."

In [ ]:
Summarizer = pipe(texts)[0]['summary_text']

In [ ]:
Summarizer

'machine learning is a subfield of artificial intelligence that focuses on designing algorithms and statistical models that enable computers to perform tasks without explicit instructions, relying instead on patterns and inference drawn from data . supervised learning involves training a model on labeled data so that it can predict outcomes for unseen inputs, with algorithms ranging from linear regression, logistic regression, support vector machines, decision trees, random forests, gradient boosting machines, to more complex neural networks .'

In [ ]:
NER = model(Summarizer)
pd.DataFrame(NER)

,entity_group,score,word,start,end
0,PRD,0.930839,linear,389,396
1,PRD,0.463518,ression,400,407
2,PRD,0.675208,logistic regression,408,428
3,PRD,0.849350,support vector machines,429,453
4,PRD,0.797734,trees,463,469
5,PRD,0.531521,random forests,470,485
6,PRD,0.727812,gr,486,489
7,PRD,0.817047,ent boosting machines,492,513
8,PRD,0.573968,ne,530,533
